In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import json
import matplotlib.image as mpimg
from matplotlib.patches import ConnectionPatch
import numpy as np
import os.path as osp
import random
from tqdm import tqdm
from scipy.spatial import distance


In [2]:
jpath_list = glob.glob("/home/alien_yhl/results/results_img_align/all_6w/*.json")
print(len(jpath_list))                

0


In [3]:
def draw_pts(pts = [([300,300], [253,313])]):  #(tem_pt , sam_pt)
    for i, pt in enumerate(pts):
        #change np.image coords to plt.plot coords
        # pt[0][1], pt[0][0] = pt[0][0], pt[0][1]
        # pt[1][1], pt[1][0] = pt[1][0], pt[1][1]

        ax[0].plot(pt[0][0], pt[0][1], 'r.')
        ax[0].text(pt[0][0], pt[0][1]*0.97, str(i+1) )
        ax[1].plot(pt[1][0], pt[1][1], 'r.')
        ax[1].text(pt[1][0], pt[1][1]*0.97, str(i+1) )
        
        con = ConnectionPatch(pt[0], pt[1], coordsA = "data", coordsB = "data",axesA= ax[0], axesB = ax[1])
        fig.add_artist(con)
def find_mpt(tem_pt, H):
    #change np.image coords to plt.plot coords
    # tem_pt[0], tem_pt[1] = tem_pt[1], tem_pt[0]
    
    _tem_pt = list(tem_pt)
    _tem_pt.append(1)
    _tem_pt = np.array(_tem_pt)
    H = np.array(H)
    # tem_pt = tem_pt.reshape(-1,1)
    sam_pt = H @ _tem_pt
    sam_pt = list(sam_pt[0:2]/sam_pt[2])
    return (tem_pt, sam_pt)

def find_gt_template(id):
    jpath = glob.glob(f"/data/home/yanghanlong/data/image_retrieval/anno_cleaned/{id}.json")
    with open(jpath[0], 'r') as f:
        dt = json.load(f)
    tem_path = dt["gt_template_image"]
    if tem_path == "":
        return (-1, tem_path)
    else:
        return (cv2.imread(osp.join("/data/home/yanghanlong/data/image_retrieval/template_image", tem_path)), tem_path)
    
def find_gt_index(data):
    for k in data["alignment_results"]:
        if data["alignment_results"][k]["isGroundTruthTemplate"] == 1:
            return k
    return -1
def find_gt_H(data, k):
    return data["alignment_results"][k]["alignment_params"]["H"]

def pt_scale2ori( mpts,data, k):
    tem_scale = data["alignment_results"][k]["alignment_params"]["template_xy_scales"]
    sam_scale = data["alignment_results"][k]["alignment_params"]["sample_xy_scales"]
    _mpts = []
    for pt in mpts:
        pt[0][0] /= tem_scale[0]
        pt[0][1] /= tem_scale[1]
        pt[1][0] /= sam_scale[0]
        pt[1][1] /= sam_scale[1]
        _mpts.append(pt)
    return _mpts

def find_pts_with_threshold(img):
    pixel_list = []
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    for y in range(thresh.shape[0]):
        for x in range(thresh.shape[1]):
            if thresh[y, x] == 0:
                pixel_list.append([x, y])
    return pixel_list



    

In [4]:
def find_tem_pts(tem, num_pts, dxy = [320, 240]):
    global H
    array_H = np.array(H)
    inverse_H = np.linalg.inv(array_H)

    black_pixels = []
    # black_pixels = [[x+dxy[0], y+ dxy[1]] for x, y in find_pts_with_threshold(tem)]
    for i in range(0, tem.shape[0]-1, 1):
        for j in range(0, tem.shape[1]-1, 1):
            if tem[i][j].sum() < 100:
                black_pixels.append([j+dxy[0], i+dxy[1]])
            else:
                pass
                if len(black_pixels) > 2:
                    if black_pixels[-1][0] - black_pixels[-2][0] == 2:
                        black_pixels.pop()
    _black_pixels = list(black_pixels)
    for  b_pixel in _black_pixels:
        _tem_pt = find_mpt(b_pixel, H)[1]
        if (0 < _tem_pt[0] < 478) and (0 < _tem_pt[1] < 638): #delete pts out of bound
            continue
        else:
           black_pixels.remove(b_pixel)
    print("="*20)
    print(len(black_pixels))
    print(black_pixels[0:10])
    print("="*20)
    num_allpts = len(black_pixels)
    # k = 0
    # while num_allpts < num_pts*2:
    #     i = random.randint(0, tem.shape[0] - 1)
    #     j = random.randint(0, tem.shape[1] - 1)
    #     color = tem[i][j]
    #     if color.sum() < 10:
    #         black_pixels.append([i,j])
    #     k+=1
    #     if k > 1000000:
    #         break
    if num_allpts == 0:
        return []
    elif num_allpts < num_pts:
        if len(black_pixels)>1 and distance.euclidean(black_pixels[0], black_pixels[-1]) < 10: #check if pts too close
            black_pixels = black_pixels[0:1]
        
        try: 
            while distance.euclidean(black_pixels[0], black_pixels[1]) < 30:
                print(black_pixels)
                black_pixels.pop(0)
                print(black_pixels)
        except:
            pass
        return [find_mpt(x, H) for x in black_pixels]
    #find middle points
    new_black_pixels = []
    x = np.linspace(1, num_allpts, num_pts)
    for i in x: 
        j = i 
        
        new_black_pixels.append(black_pixels[int(j-1)])
    if distance.euclidean(new_black_pixels[0], new_black_pixels[-1]) < 30: #check if pts too close
        new_black_pixels = new_black_pixels[0:1]
    elif distance.euclidean(new_black_pixels[-2], new_black_pixels[-1]) < 30: #check if pts too close
        new_black_pixels.pop()
    try: 
        print(f"distance of pt1 and pt2: {distance.euclidean(new_black_pixels[0], new_black_pixels[1])}")
        if distance.euclidean(new_black_pixels[-2], new_black_pixels[-3]) < 30:
            new_black_pixels.pop(-2)
        while distance.euclidean(new_black_pixels[0], new_black_pixels[1]) < 30:
            print(new_black_pixels)
            new_black_pixels.pop(0)
            print(new_black_pixels)
    except:
        pass
    mpts = [find_mpt(x, H) for x in new_black_pixels]
    # print("="*20)
    # print(mpts)
    # print("="*20)
    return mpts
    # return new_black_pixels

In [25]:
#run 
for js in tqdm(jpath_list):
    with open(js, 'r') as f:
        data = json.load(f)
    im_id = data["id"]
    gt_index = find_gt_index(data)
    if gt_index == -1:
        with open("/home/alien_yhl/results/pt_map/no_gt/no_gt_id.txt", 'a') as f:
            f.write('\n')
            f.write(im_id)
        continue
    else:
        H = find_gt_H(data, gt_index)
    tem_data = find_gt_template(im_id) ####
    tem = tem_data[0]
    if tem[0].any() == -1:
        with open("/home/alien_yhl/results/pt_map/no_gt/not_find_tem.txt", 'a') as f:
            f.write('\n')
            f.write(im_id)
        continue
    o_data = {}

    tem_rgb = cv2.cvtColor(tem, cv2.COLOR_BGR2RGB)
    tem_scale = cv2.resize(tem_rgb, (480, 640))

    # find match pts and draw them
    tem_pts = find_tem_pts(tem_scale, 16)
    mpts = [find_mpt(x, H) for x in tem_pts]
    mpts = pt_scale2ori(mpts, data, gt_index)
    template_pts = [[x[0][0], x[0][1]]for x in mpts]
    sample_pts = [[x[1][0], x[1][1]]for x in mpts]
    o_data = {
        "image_id": im_id,
        "template_path":tem_data[1],
        "template_pts": template_pts,
        "sample_pts": sample_pts
    }
    with open("/home/alien_yhl/results/pt_map/results/{}.json".format(im_id), 'w') as f:
        json.dump(o_data, f)


  0%|          | 31/64887 [00:15<8:53:40,  2.03it/s] 


KeyboardInterrupt: 

In [9]:
#trial for single jsonfile
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import glob
js = glob.glob("/data/home/yanghanlong/results/iar_on_rule/cleaned_dataset/6w_warped_sam/2a53e725-066d-434a-b33f-42e68b3e271f*.json")[0]
# js = glob.glob("/data/home/yanghanlong/results/iar_on_rule/cleaned_dataset/6w_warped_sam/*.json")[465]
print(js)
with open(js, 'r') as f:
    data = json.load(f)
im_id = data["id"]
print(im_id)
gt_index = find_gt_index(data)
if gt_index == -1:
    raise ValueError("no gt template")
else:
    H = find_gt_H(data, gt_index)
tem_data = find_gt_template(im_id)
tem = tem_data[0]
if tem.any() == -1:
    raise ValueError("not find template")
    
sam = glob.glob("/data/home/yanghanlong/data/image_retrieval/image/"+im_id+".*")
sam = sam[0]
sam = cv2.imread(sam)
sam_rgb = cv2.cvtColor(sam, cv2.COLOR_BGR2RGB)
sam_scale = cv2.resize(sam_rgb, (480, 640))
tem_rgb = cv2.cvtColor(tem, cv2.COLOR_BGR2RGB)
tem_scale = cv2.resize(tem_rgb, (480, 640))
fig, ax = plt.subplots(1,2)
ax[0].imshow(tem_rgb)
ax[1].imshow(sam_rgb)
# ax[0].imshow(tem_scale)
# ax[1].imshow(sam_scale)
# find match pts and draw them
h,w = tem_scale.shape[0], tem_scale.shape[1]

mpts = find_tem_pts(tem_scale[0:h//2,0:w//2, :], 4, dxy = [0,0])

mpts += find_tem_pts(tem_scale[0:h//2,w//2+20:w, :], 4, dxy = [260,0])

mpts += find_tem_pts(tem_scale[h//2+10:h,0:w//2, :], 4, dxy = [0, 330])
mpts += find_tem_pts(tem_scale[h//2+10:h,w//2+20:w, :], 4, dxy = [260, 330])
mpts = pt_scale2ori(mpts, data, gt_index)
print(f"before sorted: {mpts}")
mpts = sorted(mpts, key=lambda x: (x[0][1], x[0][0]))
print(f"After sorted: {mpts}")
print(f"{im_id}")
print(f"totally {len(mpts)} points")
draw_pts(mpts)


IndexError: list index out of range

In [72]:

points = [(1, 3), (2, 3), (5, 6), (4, 2), (8, 4), (7, 9), (6, 8), (9, 7), (10, 12), (11, 10), (14, 15), (13, 11), (16, 13), (15, 14), (18, 16), (17, 19)]

sorted_points = sorted(points, key=lambda x: (x[1], x[0]))

print(sorted_points)

[(4, 2), (1, 3), (2, 3), (8, 4), (5, 6), (9, 7), (6, 8), (7, 9), (11, 10), (13, 11), (10, 12), (16, 13), (15, 14), (14, 15), (18, 16), (17, 19)]


H * template pt = sample pt

In [25]:
i = cv2.imread("/home/alien_yhl/data/image_retrieval/template_image/homework-prod/14822532386499809pic.jpg")
print(i.shape)

(1856, 1188, 3)


In [11]:
a=list('abcdefg')
a[1]

'b'

In [155]:
x = find_tem_pts(tem_scale,20)
print(x)


[[422, 44], [340, 56], [284, 58], [292, 58], [470, 110], [214, 144], [30, 156], [30, 158], [62, 162], [270, 272], [442, 272], [254, 274], [442, 274], [460, 274], [270, 276], [308, 276], [442, 276], [130, 292], [410, 292], [90, 310]]


In [57]:
find_gt_index(data)

-1

In [24]:
H = np.array(H)
print(H)
pt1 = [300,300,1]
pt1 = np.array(pt1)
pt2 = H @ (pt1)
print(pt2/pt2[2])

[[ 4.60098040e-01 -1.07493439e-03  1.68378239e+01]
 [-4.67383955e-02  5.32230717e-01  4.55214873e+01]
 [-6.67739262e-05 -1.04554617e-04  6.61594605e-01]]
[253.27066193 313.29141864   1.        ]


In [21]:
pt1 = np.array([198.0, 87.0])
pt2 = np.array([239.0, 37.0])




array([500, 500,   1])